In [1]:
import pandas as pd

from petsard.loader import Splitter
from petsard.synthesizer import Synthesizer

In [2]:
split = Splitter(
    method="custom_data",
    filepath={
        "ori": "benchmark/adult-income_ori.csv",
        "control": "benchmark/adult-income_control.csv",
    },
)
split_data, split_meta, exist_train_index = split.split()


syn = Synthesizer(
    method="custom_data",
    filepath="benchmark/adult-income_syn.csv",
)
syn.create()
syn_data: pd.DataFrame = syn.fit_sample(data=None)

eval_dict: dict[str, pd.DataFrame] = {
    "ori": split_data[1]["train"],
    "control": split_data[1]["validation"],
    "syn": syn_data,
}
# eval_dict = {
#     # key: df.drop(columns=['fnlwgt']) for key, df in eval_dict.items()
#     key: df[["fnlwgt", "age", "education", "relationship"]]
#     for key, df in eval_dict.items()  # , "native-country" , "occupation"
# }

In [3]:
from typing import Any

from demo.test.mpuccs import MPUCCs

# 預設為 None
precision: int | float | dict[str, int | float] = None
# {
#     'age': 1,
#     'fnlwgt': 100,
# }
max_combination_size: int = None


config: dict[str, Any] = {
    # "precision": precision,
    # "max_combination_size": max_combination_size,
    "n_cols": 3,
}


eval = MPUCCs(config=config)
eval_result: dict[str, pd.DataFrame] = eval.eval(data=eval_dict)

eval_result["details"]

Field 15/15 - Processing capital-loss combinations: 100%|██████████| 455/455 [00:38<00:00, 11.77combo/s] 


,combo_size,syn_idx,field_combo,value_combo,ori_idx
0,3,490,"('fnlwgt', 'age', 'native-country')","(214503, 43, 'United-States')",118
1,3,1037,"('fnlwgt', 'age', 'native-country')","(129814, 28, 'United-States')",2792
2,3,1203,"('fnlwgt', 'age', 'native-country')","(232356, 33, 'United-States')",29587
3,3,1320,"('fnlwgt', 'age', 'native-country')","(142856, 49, 'United-States')",19975
4,3,1675,"('fnlwgt', 'age', 'native-country')","(340899, 33, 'United-States')",8920
...,...,...,...,...,...
7994,3,21374,"('fnlwgt', 'native-country', 'capital-loss')","(162741, 'United-States', 0)",25437
7995,3,9902,"('age', 'native-country', 'capital-loss')","(53, 'Laos', 0)",17750
7996,3,8791,"('fnlwgt', 'education', 'capital-loss')","(230467, 'Bachelors', 0)",11555
7997,3,16943,"('fnlwgt', 'education', 'capital-loss')","(237868, 'HS-grad', 0)",226


In [4]:
eval_result["global"]

,weighted_singling_out_risk,singling_out_risk,ori_total_cnt,ori_dup_cnt,syn_total_cnt,syn_dup_cnt,syn_mpuccs_collision_cnt,weighted_syn_mpuccs_collision_cnt
0,0.051230,0.204919,39073,38,39073,0,7999,1999.750000


In [5]:
eval_result["tree"]

,check_order,combo_size,field_combo,base_combo,base_is_pruned,combo_entropy,base_entropy,entropy_gain,is_pruned,mpuccs_cnt,mpuccs_collision_cnt,field_weighted,total_weighted,weighted_mpuccs_collision_cnt
0,1,3,"('fnlwgt', 'age', 'native-country')",None,None,0.999509,None,None,None,38900,54,0.25,0.25,13.50
1,2,3,"('fnlwgt', 'age', 'hours-per-week')",None,None,0.999944,None,None,None,39037,3,0.25,0.25,0.75
2,3,3,"('fnlwgt', 'native-country', 'hours-per-week')",None,None,0.998363,None,None,None,38684,143,0.25,0.25,35.75
3,4,3,"('age', 'native-country', 'hours-per-week')",None,None,0.779150,None,None,None,2542,146,0.25,0.25,36.50
4,5,3,"('fnlwgt', 'age', 'occupation')",None,None,0.999951,None,None,None,39033,9,0.25,0.25,2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,451,3,"('marital-status', 'capital-gain', 'capital-lo...",None,None,0.560011,None,None,None,0,0,0.25,0.25,0.00
451,452,3,"('relationship', 'capital-gain', 'capital-loss')",None,None,0.733218,None,None,None,0,0,0.25,0.25,0.00
452,453,3,"('race', 'capital-gain', 'capital-loss')",None,None,0.197532,None,None,None,0,0,0.25,0.25,0.00
453,454,3,"('gender', 'capital-gain', 'capital-loss')",None,None,0.841951,None,None,None,0,0,0.25,0.25,0.00


In [3]:
from typing import Any

from demo.test.mpuccs import MPUCCs

eval = MPUCCs(
    config={
        # "n_cols": [1, 3],
    }
)
eval_result: dict[str, pd.DataFrame] = eval.eval(data=eval_dict)

eval_result["details"]

KeyboardInterrupt: 

In [ ]:
len(set(eval_result["details"]["syn_idx"]))

4640

In [ ]:
eval_result["details"][
    eval_result["details"].groupby("ori_idx")["ori_idx"].transform("count") > 1
].sort_values(by="ori_idx")

,combo_size,syn_idx,field_combo,value_combo,ori_idx
3383,3,5325,"('hours-per-week', 'occupation', 'educational-...","(47, 'Tech-support', 13)",17
1891,3,8524,"('age', 'hours-per-week', 'occupation')","(34, 47, 'Tech-support')",17
4290,3,22180,"('native-country', 'educational-num', 'relatio...","('Peru', 13, 'Not-in-family')",20
4235,3,12616,"('native-country', 'education', 'relationship')","('Peru', 'Bachelors', 'Not-in-family')",20
4480,3,5907,"('hours-per-week', 'occupation', 'gender')","(50, 'Priv-house-serv', 'Male')",30
...,...,...,...,...,...
3386,3,8404,"('hours-per-week', 'occupation', 'educational-...","(42, '?', 5)",38629
3660,3,5586,"('native-country', 'education', 'workclass')","('China', 'Bachelors', 'Local-gov')",38728
3749,3,33337,"('native-country', 'educational-num', 'workcla...","('China', 13, 'Local-gov')",38728
1732,3,14015,"('fnlwgt', 'native-country', 'occupation')","(143533, 'United-States', 'Other-service')",38902


In [10]:
len(set(eval_result["details"]["ori_idx"]))

4024

In [7]:
eval_result["tree"]

,check_order,combo_size,field_combo,base_combo,base_is_pruned,combo_entropy,base_entropy,entropy_gain,is_pruned,puccs_cnt,puccs_collision_cnt,entropy_weighted,field_weighted,total_weighted,weighted_puccs_collision_cnt
0,1,1,"('fnlwgt',)",None,None,0.976751,NaN,NaN,None,34469,1576,0.023249,1.00,1.00,1576.00
1,2,1,"('age',)",None,None,0.908505,NaN,NaN,None,0,0,0.091495,1.00,1.00,0.00
2,3,1,"('native-country',)",None,None,0.054488,NaN,NaN,None,0,0,0.945512,1.00,1.00,0.00
3,4,3,"('fnlwgt', 'age', 'native-country')","('fnlwgt',)",False,0.999509,0.976751,0.022758,False,38900,33,0.000491,0.25,0.25,8.25
4,5,1,"('hours-per-week',)",None,None,0.851400,NaN,NaN,None,0,0,0.148600,1.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,466,3,"('marital-status', 'capital-gain', 'capital-lo...","('marital-status',)",False,0.560011,0.560011,0.000000,True,0,0,0.439989,0.25,0.25,0.00
466,467,3,"('relationship', 'capital-gain', 'capital-loss')","('relationship',)",False,0.733218,0.733218,0.000000,True,0,0,0.266782,0.25,0.25,0.00
467,468,3,"('race', 'capital-gain', 'capital-loss')","('race',)",False,0.197532,0.197532,0.000000,True,0,0,0.802468,0.25,0.25,0.00
468,469,3,"('gender', 'capital-gain', 'capital-loss')","('gender',)",False,0.841951,0.841951,0.000000,True,0,0,0.158049,0.25,0.25,0.00


In [ ]:
eval_result["tree"].loc[
    (eval_result["tree"]["puccs_collision_cnt"] > 0)
    & (eval_result["tree"]["combo_entropy"] < 0.6),
    :,
].head(20)

,check_order,combo_size,field_combo,base_combo,base_is_pruned,combo_entropy,base_entropy,entropy_gain,is_pruned,entropy_weighted,field_weighted,total_weighted,puccs_cnt,puccs_collision_cnt,weighted_puccs_collision_cnt
21,22,3,"('native-country', 'hours-per-week', 'education')","('native-country',)",False,0.561770,0.054488,0.507282,False,0.438230,0.25,0.109557,1133,64,7.011674
24,25,3,"('native-country', 'occupation', 'education')","('native-country',)",False,0.547989,0.054488,0.493501,False,0.452011,0.25,0.113003,850,156,17.628428
35,36,3,"('native-country', 'occupation', 'educational-...","('native-country',)",False,0.594641,0.054488,0.540153,False,0.405359,0.25,0.101340,1016,141,14.288890
39,40,3,"('native-country', 'education', 'educational-n...","('native-country',)",False,0.507362,0.054488,0.452874,False,0.492638,0.25,0.123160,785,4,0.492638
48,49,3,"('native-country', 'hours-per-week', 'workclass')","('native-country',)",False,0.472047,0.054488,0.417559,False,0.527953,0.25,0.131988,675,29,3.827659
51,52,3,"('native-country', 'occupation', 'workclass')","('native-country',)",False,0.445913,0.054488,0.391425,False,0.554087,0.25,0.138522,582,61,8.449823
55,56,3,"('native-country', 'education', 'workclass')","('native-country',)",False,0.341399,0.054488,0.286910,False,0.658601,0.25,0.164650,491,47,7.738567
60,61,3,"('native-country', 'educational-num', 'workcla...","('native-country',)",False,0.414094,0.054488,0.359606,False,0.585906,0.25,0.146476,509,43,6.298487
70,71,3,"('native-country', 'hours-per-week', 'marital-...","('native-country',)",False,0.527723,0.054488,0.473234,False,0.472277,0.25,0.118069,686,27,3.187872
73,74,3,"('native-country', 'occupation', 'marital-stat...","('native-country',)",False,0.499539,0.054488,0.445050,False,0.500461,0.25,0.125115,590,75,9.383649


In [8]:
eval_result["global"]

,singling_out_risk,ori_total_cnt,ori_dup_cnt,syn_total_cnt,syn_dup_cnt,syn_puccs_collision_cnt,weighted_syn_puccs_collision_cnt
0,0.005380,39073,38,39073,0,4640,210.015691


## 參數設計

## 檢查

In [ ]:
import math
from collections import Counter, defaultdict
from typing import Any, Optional

from scipy.stats import entropy
from tqdm import tqdm


def _main_pbar_update(
    main_pbar: tqdm,
    type: str,  # 'loop-start' or 'loop-end'
    main_pbar_info: dict[str, Any],
    combo_num: int = None,
    column_combos: frozenset[str] = None,
    singlingout_syn_indices_num: int = 0,
    syn_n_rows: int = 0,
) -> dict[str, Any]:
    add_column: str = main_pbar_info.get("add_column", None)
    column_names: str = main_pbar_info.get("column_names", None)
    source_combos_num: int = main_pbar_info.get("source_combos_num", None)
    dest_combos_num: int = main_pbar_info.get("dest_combos_num", None)
    all_combos_num: int = main_pbar_info.get("all_combos_num", None)
    if type == "loop-start":
        if combo_num is None or column_combos is None:
            raise ValueError(
                "combo_num/column_combos cannot be None when type is 'loop-start'"
            )

        if combo_num == 1:
            source_combos_num += 1
            dest_combos_num = 0
            all_combos_num = sum(
                [math.comb(source_combos_num - 1, i) for i in range(source_combos_num)]
            )
        dest_combos_num += 1
        main_pbar.set_description_str(
            f"現正處理: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )
    elif type == "loop-end":
        main_pbar.update(1)
        main_pbar.set_description_str(
            f"處理完畢: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )

    return {
        "add_column": add_column,
        "source_combos_num": source_combos_num,
        "dest_combos_num": dest_combos_num,
        "all_combos_num": all_combos_num,
    }


def _unique_and_colliding(
    column_combos: tuple[str],
    ori_data_subsets: pd.DataFrame,
    syn_combo_counter: Counter,
    counted_syn_column_combos: set[tuple[str]],
    singlingout_syn_indices: set[int],
    singlingout_syn_indices_num: int,
) -> tuple[
    bool,
    Optional[dict[str, Any]],
    Counter,
    set[tuple[str]],
    set[int],
    int,
]:
    # 只保留出現一次的組合
    all_syn_combos_count: int = len(syn_combo_counter)
    updated_syn_combo_counter: Counter = {
        k: v for k, v in syn_combo_counter.items() if v == 1
    }

    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 在原始資料計算計數，並只保留出現一次的組合
    ori_combo_counter = Counter(map(tuple, ori_data_subsets.values))
    ori_combo_counter = {k: v for k, v in ori_combo_counter.items() if v == 1}

    # 更新合成資料計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in ori_combo_counter
    }
    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 此時再來判斷本欄位組合，是否均已被成功碰撞（所有值域組合均唯一、且成功碰撞到唯一原始資料）
    if len(updated_syn_combo_counter) == all_syn_combos_count:
        counted_syn_column_combos.add(column_combos)
        print(f"欄位組合剪枝加入 {column_combos}")

    # 存儲每個組合值對應的行索引
    syn_combo_indices: defaultdict = defaultdict(int)
    for idx, row in syn_data_subsets.iterrows():
        # 同步跳過已經計數過的行
        if idx not in singlingout_syn_indices:
            value_combo = tuple(row)
            # 如果未被跳過、也在本 column_combos 的計數中是唯一
            if value_combo in updated_syn_combo_counter:
                # 納入本 column_combos 的索引
                syn_combo_indices[value_combo] = idx
                # 並添加進已計數
                singlingout_syn_indices.add(idx)
                singlingout_syn_indices_num += 1

    # 如果篩選完沒有合成索引，則跳過本 column_combos
    if not syn_combo_indices:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 反向將未被跳過的行索引，同步到合成資料的唯一計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in syn_combo_indices
    }
    # 再同步到原始資料的唯一計數
    ori_combo_counter = {
        k: v for k, v in ori_combo_counter.items() if k in syn_combo_indices
    }

    # 最後再紀錄原始資料的行索引
    ori_combo_indices: defaultdict = defaultdict(int)
    for idx, row in ori_data_subsets.iterrows():
        value_combo = tuple(row)
        if value_combo in ori_combo_counter:
            ori_combo_indices[value_combo] = idx

    return (
        True,
        {
            "unique_combos_count": len(syn_combo_indices),
            "unique_combos_indices": {
                id: {
                    "ori": ori_combo_indices.get(id),
                    "syn": syn_combo_indices.get(id),
                }
                for id in set(ori_combo_indices) | set(syn_combo_indices)
            },
        },
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )


def _get_conditional_entropy(
    pUCC_indices: set[int],
    syn_data: pd.DataFrame,
    syn_n_rows: int,
    add_column: str,
) -> float:
    # cnt_pUCC: int = len(pUCC_indices)
    cnt_pUCC: int = len(pUCC_indices)
    cnt_non_pUCC: int = syn_n_rows - cnt_pUCC

    # 2. 計算概率 p(X = mpUCC) 和 p(X ≠ mpUCC)
    p_pUCC: float = len(pUCC_indices) / syn_n_rows
    p_non_pUCC: float = 1 - p_pUCC

    # 3. 計算條件熵 H(Y|X = mpUCC)
    if cnt_pUCC > 0:
        y_given_pUCC = (
            syn_data.loc[list(pUCC_indices), add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_pUCC = entropy(y_given_pUCC)
    else:
        H_Y_given_pUCC = 0

    # 4. 計算條件熵 H(Y|X ≠ mpUCC)
    if cnt_non_pUCC > 0:
        non_pUCC_indices = ~syn_data.index.isin(pUCC_indices)
        y_given_non_pUCC = (
            syn_data.loc[non_pUCC_indices, add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_non_pUCC = entropy(y_given_non_pUCC)
    else:
        H_Y_given_non_pUCC = 0

    # 5. 計算總條件熵 H(Y|X) = p(X = mpUCC) * H(Y|X = mpUCC) + p(X ≠ mpUCC) * H(Y|X ≠ mpUCC)
    H_Y_given_X = p_pUCC * H_Y_given_pUCC + p_non_pUCC * H_Y_given_non_pUCC
    return H_Y_given_X


# 計算筆數
syn_n_rows: int = syn_data.shape[0]
# 計算總組合數
total_combinations_num: int = len(sorted_all_combinations)
# 設置主進度條
main_pbar = tqdm(
    total=total_combinations_num,
    desc="全部進度",
    bar_format="{desc}",
    position=0,
    leave=True,
)


main_pbar_info: dict[str, Any] = {
    "add_column": None,
    "column_names": None,
    "source_combos_num": 0,
    "dest_combos_num": 0,
    "all_combos_num": 0,
}
singlingout_syn_indices_num = 0
counted_syn_column_combos: set[tuple[str]] = set()
singlingout_syn_indices: set[int] = set()
singlingout_syn_indices_num: int = 0

results: dict[str, dict[str, Any]] = {}
for column_combos, source_column_combos in sorted_all_combinations.items():
    combo_num: int = len(column_combos)
    column_names: str = ", ".join(str(col) for col in column_combos)
    main_pbar_info["column_names"] = column_names
    results[column_combos] = {}

    # 捕捉本批新增的欄位
    if combo_num == 1:
        add_column: str = next(iter(column_combos))
        main_pbar_info["add_column"] = add_column

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-start",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # 各種早停規則
    skip_tag: bool = False
    break_tag: bool = False
    # stop1. 跳過已被
    if results.get(source_column_combos, {}).get("pruning", False):
        print(f"互資訊無增長剪枝觸發，跳過 {column_names}")
        results[column_combos]["pruning"] = True  # 持續剪枝
        skip_tag = True
    # stop2. 終止於所有行都已被唯一計數
    if singlingout_syn_indices_num == syn_n_rows:
        print(
            f"所有行已被指認剪枝觸發，終止於 {', '.join(str(col) for col in source_column_combos)}"
        )
        break_tag = True
    # stop3. 跳過已經唯一計數子集的超集
    if any(
        set(counted_combo).issubset(column_combos)
        for counted_combo in counted_syn_column_combos
    ):
        print(f"欄位組合剪枝觸發，跳過 {column_names}")
        skip_tag = True

    if break_tag:
        break
    if skip_tag:
        main_pbar.update(1)
        continue

    # 整理資料
    ori_data_subsets: pd.DataFrame = ori_data.loc[:, column_combos]
    syn_data_subsets: pd.DataFrame = syn_data.loc[:, column_combos]

    # 1. 計算欄位組合的熵 H(XY)
    syn_combo_counter: Counter = Counter(map(tuple, syn_data_subsets.values))
    column_combo_entropy: float = entropy(
        [(count / syn_n_rows) for count in syn_combo_counter.values()]
    )
    results[column_combos]["column_combo_entropy"] = column_combo_entropy

    # > 唯一且碰撞
    (
        unique_colliding_sts,
        unique_syn_combos_info,
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    ) = _unique_and_colliding(
        column_combos,
        ori_data_subsets,
        syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )
    # 如果沒有唯一且碰撞的組合，則跳過
    if not unique_colliding_sts:
        main_pbar.update(1)
        continue
    results[column_combos].update(unique_syn_combos_info)

    # 捕捉本次合成資料的行索引
    append_syn_combos_indices: set[int] = {
        idx
        for indices_pair in results[column_combos]["unique_combos_indices"].values()
        for data_type, idx in indices_pair.items()
        if data_type == "ori"
    }
    # 並添加上累積的行索引
    historical_syn_combos_indices: set[int] = results.get(source_column_combos, {}).get(
        "append_syn_combos_indices", set()
    )
    append_syn_combos_indices = (
        append_syn_combos_indices | historical_syn_combos_indices
    )
    results[column_combos]["append_syn_combos_indices"] = append_syn_combos_indices

    if combo_num >= 2:
        # 2. 條件熵 H(Y|X) = Σ p(X = x)*H(Y | X = x)
        conditional_entropy: float = _get_conditional_entropy(
            pUCC_indices=historical_syn_combos_indices,
            syn_data=syn_data_subsets,
            syn_n_rows=syn_n_rows,
            add_column=add_column,
        )

        # 3. 互資訊 I(X; Y)（實際熵 - 條件熵）
        mutual_information: float = column_combo_entropy - conditional_entropy

        # 4. 剪枝：如果互資訊小於等於 0，則繼承自這個欄位組合的欄位不需要繼續執行
        pruning: bool = True if mutual_information <= 0 else False

        results[column_combos].update(
            {
                "conditional_entropy": conditional_entropy,
                "mutual_information": mutual_information,
                "pruning": pruning,
            }
        )

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-end",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # print(column_combos, type(column_combos))
    if set(
        (
            "native-country",
            "fnlwgt",
            "age",
        )
    ) == set(column_combos):
        break

    if add_column == "occupation":
        break

全部進度

In [ ]:
counted_syn_column_combos

set()

In [ ]:
{
    k: {
        key: value
        for key, value in v.items()
        if key
        not in [
            "unique_combos_indices",
            "append_syn_combos_indices",
        ]
    }
    for k, v in results.items()
}

{}

In [ ]:
from collections import Counter
from typing import Any, Optional

from scipy.stats import entropy
from tqdm import tqdm


def _main_pbar_update(
    main_pbar: tqdm,
    type: str,  # 'loop-start' or 'loop-end'
    main_pbar_info: dict[str, Any],
    combo_num: int = None,
    column_combos: frozenset[str] = None,
    singlingout_syn_indices_num: int = 0,
    syn_n_rows: int = 0,
) -> dict[str, Any]:
    add_column: str = main_pbar_info.get("add_column", None)
    column_names: str = main_pbar_info.get("column_names", None)
    source_combos_num: int = main_pbar_info.get("source_combos_num", None)
    dest_combos_num: int = main_pbar_info.get("dest_combos_num", None)
    all_combos_num: int = main_pbar_info.get("all_combos_num", None)
    if type == "loop-start":
        if combo_num is None or column_combos is None:
            raise ValueError(
                "combo_num/column_combos cannot be None when type is 'loop-start'"
            )

        if combo_num == 1:
            source_combos_num += 1
            dest_combos_num = 0
            all_combos_num = sum(
                [math.comb(source_combos_num - 1, i) for i in range(source_combos_num)]
            )
        dest_combos_num += 1
        main_pbar.set_description_str(
            f"現正處理: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )
    elif type == "loop-end":
        main_pbar.update(1)
        main_pbar.set_description_str(
            f"處理完畢: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )

    return {
        "add_column": add_column,
        "source_combos_num": source_combos_num,
        "dest_combos_num": dest_combos_num,
        "all_combos_num": all_combos_num,
    }


def _unique_and_colliding(
    column_combos: tuple[str],
    ori_data_subsets: pd.DataFrame,
    syn_combo_counter: Counter,
    counted_syn_column_combos: set[tuple[str]],
    singlingout_syn_indices: set[int],
    singlingout_syn_indices_num: int,
) -> tuple[
    bool,
    Optional[dict[str, Any]],
    Counter,
    set[tuple[str]],
    set[int],
    int,
]:
    # 只保留出現一次的組合
    all_syn_combos_count: int = len(syn_combo_counter)
    updated_syn_combo_counter: Counter = {
        k: v for k, v in syn_combo_counter.items() if v == 1
    }

    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 在原始資料計算計數，並只保留出現一次的組合
    ori_combo_counter = Counter(map(tuple, ori_data_subsets.values))
    ori_combo_counter = {k: v for k, v in ori_combo_counter.items() if v == 1}

    # 更新合成資料計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in ori_combo_counter
    }
    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 此時再來判斷本欄位組合，是否均已被成功碰撞（所有值域組合均唯一、且成功碰撞到唯一原始資料）
    if len(updated_syn_combo_counter) == all_syn_combos_count:
        counted_syn_column_combos.add(column_combos)
        print(f"欄位組合剪枝加入 {column_combos}")

    # 存儲每個組合值對應的行索引
    syn_combo_indices: defaultdict = defaultdict(int)
    for idx, row in syn_data_subsets.iterrows():
        # 同步跳過已經計數過的行
        if idx not in singlingout_syn_indices:
            value_combo = tuple(row)
            # 如果未被跳過、也在本 column_combos 的計數中是唯一
            if value_combo in updated_syn_combo_counter:
                # 納入本 column_combos 的索引
                syn_combo_indices[value_combo] = idx
                # 並添加進已計數
                singlingout_syn_indices.add(idx)
                singlingout_syn_indices_num += 1

    # 如果篩選完沒有合成索引，則跳過本 column_combos
    if not syn_combo_indices:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 反向將未被跳過的行索引，同步到合成資料的唯一計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in syn_combo_indices
    }
    # 再同步到原始資料的唯一計數
    ori_combo_counter = {
        k: v for k, v in ori_combo_counter.items() if k in syn_combo_indices
    }

    # 最後再紀錄原始資料的行索引
    ori_combo_indices: defaultdict = defaultdict(int)
    for idx, row in ori_data_subsets.iterrows():
        value_combo = tuple(row)
        if value_combo in ori_combo_counter:
            ori_combo_indices[value_combo] = idx

    return (
        True,
        {
            "unique_combos_count": len(syn_combo_indices),
            "unique_combos_indices": {
                id: {
                    "ori": ori_combo_indices.get(id),
                    "syn": syn_combo_indices.get(id),
                }
                for id in set(ori_combo_indices) | set(syn_combo_indices)
            },
        },
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )


def _get_conditional_entropy(
    pUCC_indices: set[int],
    syn_data: pd.DataFrame,
    syn_n_rows: int,
    add_column: str,
) -> float:
    # cnt_pUCC: int = len(pUCC_indices)
    cnt_pUCC: int = len(pUCC_indices)
    cnt_non_pUCC: int = syn_n_rows - cnt_pUCC

    # 2. 計算概率 p(X = mpUCC) 和 p(X ≠ mpUCC)
    p_pUCC: float = len(pUCC_indices) / syn_n_rows
    p_non_pUCC: float = 1 - p_pUCC

    # 3. 計算條件熵 H(Y|X = mpUCC)
    if cnt_pUCC > 0:
        y_given_pUCC = (
            syn_data.loc[list(pUCC_indices), add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_pUCC = entropy(y_given_pUCC)
    else:
        H_Y_given_pUCC = 0

    # 4. 計算條件熵 H(Y|X ≠ mpUCC)
    if cnt_non_pUCC > 0:
        non_pUCC_indices = ~syn_data.index.isin(pUCC_indices)
        y_given_non_pUCC = (
            syn_data.loc[non_pUCC_indices, add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_non_pUCC = entropy(y_given_non_pUCC)
    else:
        H_Y_given_non_pUCC = 0

    # 5. 計算總條件熵 H(Y|X) = p(X = mpUCC) * H(Y|X = mpUCC) + p(X ≠ mpUCC) * H(Y|X ≠ mpUCC)
    H_Y_given_X = p_pUCC * H_Y_given_pUCC + p_non_pUCC * H_Y_given_non_pUCC
    return H_Y_given_X


# 計算筆數
syn_n_rows: int = syn_data.shape[0]
# 計算總組合數
total_combinations_num: int = len(sorted_all_combinations)
# 設置主進度條
main_pbar = tqdm(
    total=total_combinations_num,
    desc="全部進度",
    bar_format="{desc}",
    position=0,
    leave=True,
)


main_pbar_info: dict[str, Any] = {
    "add_column": None,
    "column_names": None,
    "source_combos_num": 0,
    "dest_combos_num": 0,
    "all_combos_num": 0,
}
singlingout_syn_indices_num = 0
counted_syn_column_combos: set[tuple[str]] = set()
singlingout_syn_indices: set[int] = set()
singlingout_syn_indices_num: int = 0

results: dict[str, dict[str, Any]] = {}
for column_combos, source_column_combos in sorted_all_combinations.items():
    combo_num: int = len(column_combos)
    column_names: str = ", ".join(str(col) for col in column_combos)
    main_pbar_info["column_names"] = column_names
    results[column_combos] = {}

    # 捕捉本批新增的欄位
    if combo_num == 1:
        add_column: str = next(iter(column_combos))
        main_pbar_info["add_column"] = add_column

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-start",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # 各種早停規則
    skip_tag: bool = False
    break_tag: bool = False
    # stop1. 跳過已被
    if results.get(source_column_combos, {}).get("pruning", False):
        print(f"互資訊無增長剪枝觸發，跳過 {column_names}")
        results[column_combos]["pruning"] = True  # 持續剪枝
        skip_tag = True
    # stop2. 終止於所有行都已被唯一計數
    if singlingout_syn_indices_num == syn_n_rows:
        print(
            f"所有行已被指認剪枝觸發，終止於 {', '.join(str(col) for col in source_column_combos)}"
        )
        break_tag = True
    # stop3. 跳過已經唯一計數子集的超集
    if any(
        set(counted_combo).issubset(column_combos)
        for counted_combo in counted_syn_column_combos
    ):
        print(f"欄位組合剪枝觸發，跳過 {column_names}")
        skip_tag = True

    if break_tag:
        break
    if skip_tag:
        main_pbar.update(1)
        continue

    # 整理資料
    ori_data_subsets: pd.DataFrame = ori_data.loc[:, column_combos]
    syn_data_subsets: pd.DataFrame = syn_data.loc[:, column_combos]

    # 1. 計算欄位組合的熵 H(XY)
    syn_combo_counter: Counter = Counter(map(tuple, syn_data_subsets.values))
    column_combo_entropy: float = entropy(
        [(count / syn_n_rows) for count in syn_combo_counter.values()]
    )
    results[column_combos]["column_combo_entropy"] = column_combo_entropy

    # > 唯一且碰撞
    (
        unique_colliding_sts,
        unique_syn_combos_info,
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    ) = _unique_and_colliding(
        column_combos,
        ori_data_subsets,
        syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )
    # 如果沒有唯一且碰撞的組合，則跳過
    if not unique_colliding_sts:
        main_pbar.update(1)
        continue
    results[column_combos].update(unique_syn_combos_info)

    # 捕捉本次合成資料的行索引
    append_syn_combos_indices: set[int] = {
        idx
        for indices_pair in results[column_combos]["unique_combos_indices"].values()
        for data_type, idx in indices_pair.items()
        if data_type == "ori"
    }
    # 並添加上累積的行索引
    historical_syn_combos_indices: set[int] = results.get(source_column_combos, {}).get(
        "append_syn_combos_indices", set()
    )
    append_syn_combos_indices = (
        append_syn_combos_indices | historical_syn_combos_indices
    )
    results[column_combos]["append_syn_combos_indices"] = append_syn_combos_indices

    if combo_num >= 2:
        # 2. 條件熵 H(Y|X) = Σ p(X = x)*H(Y | X = x)
        conditional_entropy: float = _get_conditional_entropy(
            pUCC_indices=historical_syn_combos_indices,
            syn_data=syn_data_subsets,
            syn_n_rows=syn_n_rows,
            add_column=add_column,
        )

        # 3. 互資訊 I(X; Y)（實際熵 - 條件熵）
        mutual_information: float = column_combo_entropy - conditional_entropy

        # 4. 剪枝：如果互資訊小於等於 0，則繼承自這個欄位組合的欄位不需要繼續執行
        pruning: bool = True if mutual_information <= 0 else False

        results[column_combos].update(
            {
                "conditional_entropy": conditional_entropy,
                "mutual_information": mutual_information,
                "pruning": pruning,
            }
        )

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-end",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # print(column_combos, type(column_combos))
    if set(
        (
            "native-country",
            "fnlwgt",
            "age",
        )
    ) == set(column_combos):
        break

    if add_column == "occupation":
        break

全部進度

In [ ]:
counted_syn_column_combos

set()

In [ ]:
{
    k: {
        key: value
        for key, value in v.items()
        if key
        not in [
            "unique_combos_indices",
            "append_syn_combos_indices",
        ]
    }
    for k, v in results.items()
}

{}

In [ ]:
eval_results = eval.eval(data=eval_dict)
eval_results["details"]

,combo_size,syn_idx,field_combo,values,ori_idx
0,1,3,fnlwgt,"(208608,)",12608
1,1,108,fnlwgt,"(49308,)",18137
2,1,130,fnlwgt,"(152619,)",12581
3,1,141,fnlwgt,"(199963,)",18738
4,1,248,fnlwgt,"(100997,)",14936
...,...,...,...,...,...
1571,1,38970,fnlwgt,"(269417,)",35534
1572,1,38977,fnlwgt,"(202156,)",4272
1573,1,38994,fnlwgt,"(248708,)",29290
1574,1,39047,fnlwgt,"(158319,)",16753


In [ ]:
eval._progressive_field_search(
    sorted_columns=sorted_columns,
    syn_data=data["syn"],
    ori_data=data["ori"],
)

([{'combo_size': 1,
   'syn_idx': 3,
   'field_combo': 'fnlwgt',
   'values': '(208608,)',
   'ori_idx': 12612},
  {'combo_size': 1,
   'syn_idx': 108,
   'field_combo': 'fnlwgt',
   'values': '(49308,)',
   'ori_idx': 18147},
  {'combo_size': 1,
   'syn_idx': 130,
   'field_combo': 'fnlwgt',
   'values': '(152619,)',
   'ori_idx': 12585},
  {'combo_size': 1,
   'syn_idx': 141,
   'field_combo': 'fnlwgt',
   'values': '(199963,)',
   'ori_idx': 18748},
  {'combo_size': 1,
   'syn_idx': 248,
   'field_combo': 'fnlwgt',
   'values': '(100997,)',
   'ori_idx': 14941},
  {'combo_size': 1,
   'syn_idx': 347,
   'field_combo': 'fnlwgt',
   'values': '(272531,)',
   'ori_idx': 38168},
  {'combo_size': 1,
   'syn_idx': 361,
   'field_combo': 'fnlwgt',
   'values': '(162397,)',
   'ori_idx': 8549},
  {'combo_size': 1,
   'syn_idx': 383,
   'field_combo': 'fnlwgt',
   'values': '(176315,)',
   'ori_idx': 8003},
  {'combo_size': 1,
   'syn_idx': 420,
   'field_combo': 'fnlwgt',
   'values': '(132

In [ ]:
syn_data = data["syn"]
ori_data = data["ori"]

details_results = []
identified_indices = set()
iteration_counter = 0
# 有效組合集合 - 只有通過熵檢驗的組合才會被保留
valid_combinations = set()
# 剪枝組合集合 - 被剪枝的組合不會再被考慮
pruned_combinations = set()

max_size = len(sorted_columns)
n_cols = None

# 決定要處理的組合大小
if n_cols is None:
    target_sizes = set(range(1, max_size + 1))
elif isinstance(n_cols, int):
    target_sizes = {n_cols} if n_cols <= max_size else set()
elif isinstance(n_cols, list):
    target_sizes = {size for size in n_cols if size <= max_size}
else:
    target_sizes = set(range(1, max_size + 1))
print(target_sizes)

# 漸進式加入欄位
for field_idx, current_field in enumerate(sorted_columns):
    print(f"處理第 {field_idx + 1} 個欄位: {current_field}")

    # 當前步驟要評估的組合
    current_step_combos = []

    # 1. 單獨的當前欄位
    if 1 in target_sizes:
        single_combo = (current_field,)
        current_step_combos.append((single_combo, None))  # (組合, 基礎組合)

    # 2. 當前欄位與之前所有有效組合的結合
    print(f"當前有效組合: {valid_combinations}")
    for existing_combo in list(valid_combinations):
        new_combo = tuple(
            sorted(
                existing_combo + (current_field,),
                key=lambda x: sorted_columns.index(x),
            )
        )

        # 檢查組合大小是否符合要求
        if len(new_combo) in target_sizes and len(new_combo) <= max_size:
            current_step_combos.append((new_combo, existing_combo))

        print(f"當前步驟的組合: {existing_combo}")

    # 評估當前步驟的所有組合
    for combo, base_combo in current_step_combos:
        # 檢查是否已被剪枝
        if combo in pruned_combinations:
            continue

        # 條件熵檢查（只有在有基礎組合且基礎組合也在目標大小中時才進行）
        should_check_entropy = (
            base_combo
            and True  # self.config.get("enable_conditional_entropy_pruning",
            and len(base_combo) in target_sizes  # 關鍵：基礎組合必須也在目標大小中
        )
        # print(base_combo)
        print(should_check_entropy)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
處理第 1 個欄位: fnlwgt
當前有效組合: set()
None
處理第 2 個欄位: age
當前有效組合: set()
None
處理第 3 個欄位: native-country
當前有效組合: set()
None
處理第 4 個欄位: hours-per-week
當前有效組合: set()
None
處理第 5 個欄位: occupation
當前有效組合: set()
None
處理第 6 個欄位: education
當前有效組合: set()
None
處理第 7 個欄位: educational-num
當前有效組合: set()
None
處理第 8 個欄位: workclass
當前有效組合: set()
None
處理第 9 個欄位: marital-status
當前有效組合: set()
None
處理第 10 個欄位: relationship
當前有效組合: set()
None
處理第 11 個欄位: race
當前有效組合: set()
None
處理第 12 個欄位: gender
當前有效組合: set()
None
處理第 13 個欄位: income
當前有效組合: set()
None
處理第 14 個欄位: capital-gain
當前有效組合: set()
None
處理第 15 個欄位: capital-loss
當前有效組合: set()
None


In [ ]:
from typing import Optional

eval = MPUCCs(config=config)
syn_column_info = eval._get_column_info(eval_dict["syn"], eval.config["precision"])
sorted_columns: list[str] = [info["colname"] for info in syn_column_info]
sorted_all_combinations: dict[tuple[str], Optional[tuple[str]]] = (
    eval._get_sorted_combinations(
        sorted_columns,
        max_combination_size=eval.config["max_combination_size"],
    )
)

ori_data = eval._preprocess_data(eval_dict["ori"], syn_column_info)
syn_data = eval._preprocess_data(eval_dict["syn"], syn_column_info)

AttributeError: 'MPUCCs' object has no attribute '_get_sorted_combinations'

In [ ]:
from collections import Counter
from typing import Any, Optional

from scipy.stats import entropy
from tqdm import tqdm


def _main_pbar_update(
    main_pbar: tqdm,
    type: str,  # 'loop-start' or 'loop-end'
    main_pbar_info: dict[str, Any],
    combo_num: int = None,
    column_combos: frozenset[str] = None,
    singlingout_syn_indices_num: int = 0,
    syn_n_rows: int = 0,
) -> dict[str, Any]:
    add_column: str = main_pbar_info.get("add_column", None)
    column_names: str = main_pbar_info.get("column_names", None)
    source_combos_num: int = main_pbar_info.get("source_combos_num", None)
    dest_combos_num: int = main_pbar_info.get("dest_combos_num", None)
    all_combos_num: int = main_pbar_info.get("all_combos_num", None)
    if type == "loop-start":
        if combo_num is None or column_combos is None:
            raise ValueError(
                "combo_num/column_combos cannot be None when type is 'loop-start'"
            )

        if combo_num == 1:
            source_combos_num += 1
            dest_combos_num = 0
            all_combos_num = sum(
                [math.comb(source_combos_num - 1, i) for i in range(source_combos_num)]
            )
        dest_combos_num += 1
        main_pbar.set_description_str(
            f"現正處理: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )
    elif type == "loop-end":
        main_pbar.update(1)
        main_pbar.set_description_str(
            f"處理完畢: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )

    return {
        "add_column": add_column,
        "source_combos_num": source_combos_num,
        "dest_combos_num": dest_combos_num,
        "all_combos_num": all_combos_num,
    }


def _unique_and_colliding(
    column_combos: tuple[str],
    ori_data_subsets: pd.DataFrame,
    syn_combo_counter: Counter,
    counted_syn_column_combos: set[tuple[str]],
    singlingout_syn_indices: set[int],
    singlingout_syn_indices_num: int,
) -> tuple[
    bool,
    Optional[dict[str, Any]],
    Counter,
    set[tuple[str]],
    set[int],
    int,
]:
    # 只保留出現一次的組合
    all_syn_combos_count: int = len(syn_combo_counter)
    updated_syn_combo_counter: Counter = {
        k: v for k, v in syn_combo_counter.items() if v == 1
    }

    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 在原始資料計算計數，並只保留出現一次的組合
    ori_combo_counter = Counter(map(tuple, ori_data_subsets.values))
    ori_combo_counter = {k: v for k, v in ori_combo_counter.items() if v == 1}

    # 更新合成資料計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in ori_combo_counter
    }
    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 此時再來判斷本欄位組合，是否均已被成功碰撞（所有值域組合均唯一、且成功碰撞到唯一原始資料）
    if len(updated_syn_combo_counter) == all_syn_combos_count:
        counted_syn_column_combos.add(column_combos)
        print(f"欄位組合剪枝加入 {column_combos}")

    # 存儲每個組合值對應的行索引
    syn_combo_indices: defaultdict = defaultdict(int)
    for idx, row in syn_data_subsets.iterrows():
        # 同步跳過已經計數過的行
        if idx not in singlingout_syn_indices:
            value_combo = tuple(row)
            # 如果未被跳過、也在本 column_combos 的計數中是唯一
            if value_combo in updated_syn_combo_counter:
                # 納入本 column_combos 的索引
                syn_combo_indices[value_combo] = idx
                # 並添加進已計數
                singlingout_syn_indices.add(idx)
                singlingout_syn_indices_num += 1

    # 如果篩選完沒有合成索引，則跳過本 column_combos
    if not syn_combo_indices:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 反向將未被跳過的行索引，同步到合成資料的唯一計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in syn_combo_indices
    }
    # 再同步到原始資料的唯一計數
    ori_combo_counter = {
        k: v for k, v in ori_combo_counter.items() if k in syn_combo_indices
    }

    # 最後再紀錄原始資料的行索引
    ori_combo_indices: defaultdict = defaultdict(int)
    for idx, row in ori_data_subsets.iterrows():
        value_combo = tuple(row)
        if value_combo in ori_combo_counter:
            ori_combo_indices[value_combo] = idx

    return (
        True,
        {
            "unique_combos_count": len(syn_combo_indices),
            "unique_combos_indices": {
                id: {
                    "ori": ori_combo_indices.get(id),
                    "syn": syn_combo_indices.get(id),
                }
                for id in set(ori_combo_indices) | set(syn_combo_indices)
            },
        },
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )


def _get_conditional_entropy(
    pUCC_indices: set[int],
    syn_data: pd.DataFrame,
    syn_n_rows: int,
    add_column: str,
) -> float:
    # cnt_pUCC: int = len(pUCC_indices)
    cnt_pUCC: int = len(pUCC_indices)
    cnt_non_pUCC: int = syn_n_rows - cnt_pUCC

    # 2. 計算概率 p(X = mpUCC) 和 p(X ≠ mpUCC)
    p_pUCC: float = len(pUCC_indices) / syn_n_rows
    p_non_pUCC: float = 1 - p_pUCC

    # 3. 計算條件熵 H(Y|X = mpUCC)
    if cnt_pUCC > 0:
        y_given_pUCC = (
            syn_data.loc[list(pUCC_indices), add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_pUCC = entropy(y_given_pUCC)
    else:
        H_Y_given_pUCC = 0

    # 4. 計算條件熵 H(Y|X ≠ mpUCC)
    if cnt_non_pUCC > 0:
        non_pUCC_indices = ~syn_data.index.isin(pUCC_indices)
        y_given_non_pUCC = (
            syn_data.loc[non_pUCC_indices, add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_non_pUCC = entropy(y_given_non_pUCC)
    else:
        H_Y_given_non_pUCC = 0

    # 5. 計算總條件熵 H(Y|X) = p(X = mpUCC) * H(Y|X = mpUCC) + p(X ≠ mpUCC) * H(Y|X ≠ mpUCC)
    H_Y_given_X = p_pUCC * H_Y_given_pUCC + p_non_pUCC * H_Y_given_non_pUCC
    return H_Y_given_X


# 計算筆數
syn_n_rows: int = syn_data.shape[0]
# 計算總組合數
total_combinations_num: int = len(sorted_all_combinations)
# 設置主進度條
main_pbar = tqdm(
    total=total_combinations_num,
    desc="全部進度",
    bar_format="{desc}",
    position=0,
    leave=True,
)


main_pbar_info: dict[str, Any] = {
    "add_column": None,
    "column_names": None,
    "source_combos_num": 0,
    "dest_combos_num": 0,
    "all_combos_num": 0,
}
singlingout_syn_indices_num = 0
counted_syn_column_combos: set[tuple[str]] = set()
singlingout_syn_indices: set[int] = set()
singlingout_syn_indices_num: int = 0

results: dict[str, dict[str, Any]] = {}
for column_combos, source_column_combos in sorted_all_combinations.items():
    combo_num: int = len(column_combos)
    column_names: str = ", ".join(str(col) for col in column_combos)
    main_pbar_info["column_names"] = column_names
    results[column_combos] = {}

    # 捕捉本批新增的欄位
    if combo_num == 1:
        add_column: str = next(iter(column_combos))
        main_pbar_info["add_column"] = add_column

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-start",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # 各種早停規則
    skip_tag: bool = False
    break_tag: bool = False
    # stop1. 跳過已被
    if results.get(source_column_combos, {}).get("pruning", False):
        print(f"互資訊無增長剪枝觸發，跳過 {column_names}")
        results[column_combos]["pruning"] = True  # 持續剪枝
        skip_tag = True
    # stop2. 終止於所有行都已被唯一計數
    if singlingout_syn_indices_num == syn_n_rows:
        print(
            f"所有行已被指認剪枝觸發，終止於 {', '.join(str(col) for col in source_column_combos)}"
        )
        break_tag = True
    # stop3. 跳過已經唯一計數子集的超集
    if any(
        set(counted_combo).issubset(column_combos)
        for counted_combo in counted_syn_column_combos
    ):
        print(f"欄位組合剪枝觸發，跳過 {column_names}")
        skip_tag = True

    if break_tag:
        break
    if skip_tag:
        main_pbar.update(1)
        continue

    # 整理資料
    ori_data_subsets: pd.DataFrame = ori_data.loc[:, column_combos]
    syn_data_subsets: pd.DataFrame = syn_data.loc[:, column_combos]

    # 1. 計算欄位組合的熵 H(XY)
    syn_combo_counter: Counter = Counter(map(tuple, syn_data_subsets.values))
    column_combo_entropy: float = entropy(
        [(count / syn_n_rows) for count in syn_combo_counter.values()]
    )
    results[column_combos]["column_combo_entropy"] = column_combo_entropy

    # > 唯一且碰撞
    (
        unique_colliding_sts,
        unique_syn_combos_info,
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    ) = _unique_and_colliding(
        column_combos,
        ori_data_subsets,
        syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )
    # 如果沒有唯一且碰撞的組合，則跳過
    if not unique_colliding_sts:
        main_pbar.update(1)
        continue
    results[column_combos].update(unique_syn_combos_info)

    # 捕捉本次合成資料的行索引
    append_syn_combos_indices: set[int] = {
        idx
        for indices_pair in results[column_combos]["unique_combos_indices"].values()
        for data_type, idx in indices_pair.items()
        if data_type == "ori"
    }
    # 並添加上累積的行索引
    historical_syn_combos_indices: set[int] = results.get(source_column_combos, {}).get(
        "append_syn_combos_indices", set()
    )
    append_syn_combos_indices = (
        append_syn_combos_indices | historical_syn_combos_indices
    )
    results[column_combos]["append_syn_combos_indices"] = append_syn_combos_indices

    if combo_num >= 2:
        # 2. 條件熵 H(Y|X) = Σ p(X = x)*H(Y | X = x)
        conditional_entropy: float = _get_conditional_entropy(
            pUCC_indices=historical_syn_combos_indices,
            syn_data=syn_data_subsets,
            syn_n_rows=syn_n_rows,
            add_column=add_column,
        )

        # 3. 互資訊 I(X; Y)（實際熵 - 條件熵）
        mutual_information: float = column_combo_entropy - conditional_entropy

        # 4. 剪枝：如果互資訊小於等於 0，則繼承自這個欄位組合的欄位不需要繼續執行
        pruning: bool = True if mutual_information <= 0 else False

        results[column_combos].update(
            {
                "conditional_entropy": conditional_entropy,
                "mutual_information": mutual_information,
                "pruning": pruning,
            }
        )

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-end",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # print(column_combos, type(column_combos))
    if set(
        (
            "native-country",
            "fnlwgt",
            "age",
        )
    ) == set(column_combos):
        break

    if add_column == "occupation":
        break

全部進度

In [ ]:
counted_syn_column_combos

set()

In [ ]:
{
    k: {
        key: value
        for key, value in v.items()
        if key
        not in [
            "unique_combos_indices",
            "append_syn_combos_indices",
        ]
    }
    for k, v in results.items()
}

{}

In [ ]:
from typing import Optional

eval = MPUCCs(config=config)
syn_column_info = eval._get_column_info(eval_dict["syn"], eval.config["precision"])
sorted_columns: list[str] = [info["colname"] for info in syn_column_info]
sorted_all_combinations: dict[tuple[str], Optional[tuple[str]]] = (
    eval._get_sorted_combinations(
        sorted_columns,
        max_combination_size=eval.config["max_combination_size"],
    )
)

ori_data = eval._preprocess_data(eval_dict["ori"], syn_column_info)
syn_data = eval._preprocess_data(eval_dict["syn"], syn_column_info)

AttributeError: 'MPUCCs' object has no attribute '_get_sorted_combinations'

In [ ]:
from collections import Counter
from typing import Any, Optional

from scipy.stats import entropy
from tqdm import tqdm


def _main_pbar_update(
    main_pbar: tqdm,
    type: str,  # 'loop-start' or 'loop-end'
    main_pbar_info: dict[str, Any],
    combo_num: int = None,
    column_combos: frozenset[str] = None,
    singlingout_syn_indices_num: int = 0,
    syn_n_rows: int = 0,
) -> dict[str, Any]:
    add_column: str = main_pbar_info.get("add_column", None)
    column_names: str = main_pbar_info.get("column_names", None)
    source_combos_num: int = main_pbar_info.get("source_combos_num", None)
    dest_combos_num: int = main_pbar_info.get("dest_combos_num", None)
    all_combos_num: int = main_pbar_info.get("all_combos_num", None)
    if type == "loop-start":
        if combo_num is None or column_combos is None:
            raise ValueError(
                "combo_num/column_combos cannot be None when type is 'loop-start'"
            )

        if combo_num == 1:
            source_combos_num += 1
            dest_combos_num = 0
            all_combos_num = sum(
                [math.comb(source_combos_num - 1, i) for i in range(source_combos_num)]
            )
        dest_combos_num += 1
        main_pbar.set_description_str(
            f"現正處理: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )
    elif type == "loop-end":
        main_pbar.update(1)
        main_pbar.set_description_str(
            f"處理完畢: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )

    return {
        "add_column": add_column,
        "source_combos_num": source_combos_num,
        "dest_combos_num": dest_combos_num,
        "all_combos_num": all_combos_num,
    }


def _unique_and_colliding(
    column_combos: tuple[str],
    ori_data_subsets: pd.DataFrame,
    syn_combo_counter: Counter,
    counted_syn_column_combos: set[tuple[str]],
    singlingout_syn_indices: set[int],
    singlingout_syn_indices_num: int,
) -> tuple[
    bool,
    Optional[dict[str, Any]],
    Counter,
    set[tuple[str]],
    set[int],
    int,
]:
    # 只保留出現一次的組合
    all_syn_combos_count: int = len(syn_combo_counter)
    updated_syn_combo_counter: Counter = {
        k: v for k, v in syn_combo_counter.items() if v == 1
    }

    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 在原始資料計算計數，並只保留出現一次的組合
    ori_combo_counter = Counter(map(tuple, ori_data_subsets.values))
    ori_combo_counter = {k: v for k, v in ori_combo_counter.items() if v == 1}

    # 更新合成資料計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in ori_combo_counter
    }
    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 此時再來判斷本欄位組合，是否均已被成功碰撞（所有值域組合均唯一、且成功碰撞到唯一原始資料）
    if len(updated_syn_combo_counter) == all_syn_combos_count:
        counted_syn_column_combos.add(column_combos)
        print(f"欄位組合剪枝加入 {column_combos}")

    # 存儲每個組合值對應的行索引
    syn_combo_indices: defaultdict = defaultdict(int)
    for idx, row in syn_data_subsets.iterrows():
        # 同步跳過已經計數過的行
        if idx not in singlingout_syn_indices:
            value_combo = tuple(row)
            # 如果未被跳過、也在本 column_combos 的計數中是唯一
            if value_combo in updated_syn_combo_counter:
                # 納入本 column_combos 的索引
                syn_combo_indices[value_combo] = idx
                # 並添加進已計數
                singlingout_syn_indices.add(idx)
                singlingout_syn_indices_num += 1

    # 如果篩選完沒有合成索引，則跳過本 column_combos
    if not syn_combo_indices:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 反向將未被跳過的行索引，同步到合成資料的唯一計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in syn_combo_indices
    }
    # 再同步到原始資料的唯一計數
    ori_combo_counter = {
        k: v for k, v in ori_combo_counter.items() if k in syn_combo_indices
    }

    # 最後再紀錄原始資料的行索引
    ori_combo_indices: defaultdict = defaultdict(int)
    for idx, row in ori_data_subsets.iterrows():
        value_combo = tuple(row)
        if value_combo in ori_combo_counter:
            ori_combo_indices[value_combo] = idx

    return (
        True,
        {
            "unique_combos_count": len(syn_combo_indices),
            "unique_combos_indices": {
                id: {
                    "ori": ori_combo_indices.get(id),
                    "syn": syn_combo_indices.get(id),
                }
                for id in set(ori_combo_indices) | set(syn_combo_indices)
            },
        },
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )


def _get_conditional_entropy(
    pUCC_indices: set[int],
    syn_data: pd.DataFrame,
    syn_n_rows: int,
    add_column: str,
) -> float:
    # cnt_pUCC: int = len(pUCC_indices)
    cnt_pUCC: int = len(pUCC_indices)
    cnt_non_pUCC: int = syn_n_rows - cnt_pUCC

    # 2. 計算概率 p(X = mpUCC) 和 p(X ≠ mpUCC)
    p_pUCC: float = len(pUCC_indices) / syn_n_rows
    p_non_pUCC: float = 1 - p_pUCC

    # 3. 計算條件熵 H(Y|X = mpUCC)
    if cnt_pUCC > 0:
        y_given_pUCC = (
            syn_data.loc[list(pUCC_indices), add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_pUCC = entropy(y_given_pUCC)
    else:
        H_Y_given_pUCC = 0

    # 4. 計算條件熵 H(Y|X ≠ mpUCC)
    if cnt_non_pUCC > 0:
        non_pUCC_indices = ~syn_data.index.isin(pUCC_indices)
        y_given_non_pUCC = (
            syn_data.loc[non_pUCC_indices, add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_non_pUCC = entropy(y_given_non_pUCC)
    else:
        H_Y_given_non_pUCC = 0

    # 5. 計算總條件熵 H(Y|X) = p(X = mpUCC) * H(Y|X = mpUCC) + p(X ≠ mpUCC) * H(Y|X ≠ mpUCC)
    H_Y_given_X = p_pUCC * H_Y_given_pUCC + p_non_pUCC * H_Y_given_non_pUCC
    return H_Y_given_X


# 計算筆數
syn_n_rows: int = syn_data.shape[0]
# 計算總組合數
total_combinations_num: int = len(sorted_all_combinations)
# 設置主進度條
main_pbar = tqdm(
    total=total_combinations_num,
    desc="全部進度",
    bar_format="{desc}",
    position=0,
    leave=True,
)


main_pbar_info: dict[str, Any] = {
    "add_column": None,
    "column_names": None,
    "source_combos_num": 0,
    "dest_combos_num": 0,
    "all_combos_num": 0,
}
singlingout_syn_indices_num = 0
counted_syn_column_combos: set[tuple[str]] = set()
singlingout_syn_indices: set[int] = set()
singlingout_syn_indices_num: int = 0

results: dict[str, dict[str, Any]] = {}
for column_combos, source_column_combos in sorted_all_combinations.items():
    combo_num: int = len(column_combos)
    column_names: str = ", ".join(str(col) for col in column_combos)
    main_pbar_info["column_names"] = column_names
    results[column_combos] = {}

    # 捕捉本批新增的欄位
    if combo_num == 1:
        add_column: str = next(iter(column_combos))
        main_pbar_info["add_column"] = add_column

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-start",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # 各種早停規則
    skip_tag: bool = False
    break_tag: bool = False
    # stop1. 跳過已被
    if results.get(source_column_combos, {}).get("pruning", False):
        print(f"互資訊無增長剪枝觸發，跳過 {column_names}")
        results[column_combos]["pruning"] = True  # 持續剪枝
        skip_tag = True
    # stop2. 終止於所有行都已被唯一計數
    if singlingout_syn_indices_num == syn_n_rows:
        print(
            f"所有行已被指認剪枝觸發，終止於 {', '.join(str(col) for col in source_column_combos)}"
        )
        break_tag = True
    # stop3. 跳過已經唯一計數子集的超集
    if any(
        set(counted_combo).issubset(column_combos)
        for counted_combo in counted_syn_column_combos
    ):
        print(f"欄位組合剪枝觸發，跳過 {column_names}")
        skip_tag = True

    if break_tag:
        break
    if skip_tag:
        main_pbar.update(1)
        continue

    # 整理資料
    ori_data_subsets: pd.DataFrame = ori_data.loc[:, column_combos]
    syn_data_subsets: pd.DataFrame = syn_data.loc[:, column_combos]

    # 1. 計算欄位組合的熵 H(XY)
    syn_combo_counter: Counter = Counter(map(tuple, syn_data_subsets.values))
    column_combo_entropy: float = entropy(
        [(count / syn_n_rows) for count in syn_combo_counter.values()]
    )
    results[column_combos]["column_combo_entropy"] = column_combo_entropy

    # > 唯一且碰撞
    (
        unique_colliding_sts,
        unique_syn_combos_info,
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    ) = _unique_and_colliding(
        column_combos,
        ori_data_subsets,
        syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )
    # 如果沒有唯一且碰撞的組合，則跳過
    if not unique_colliding_sts:
        main_pbar.update(1)
        continue
    results[column_combos].update(unique_syn_combos_info)

    # 捕捉本次合成資料的行索引
    append_syn_combos_indices: set[int] = {
        idx
        for indices_pair in results[column_combos]["unique_combos_indices"].values()
        for data_type, idx in indices_pair.items()
        if data_type == "ori"
    }
    # 並添加上累積的行索引
    historical_syn_combos_indices: set[int] = results.get(source_column_combos, {}).get(
        "append_syn_combos_indices", set()
    )
    append_syn_combos_indices = (
        append_syn_combos_indices | historical_syn_combos_indices
    )
    results[column_combos]["append_syn_combos_indices"] = append_syn_combos_indices

    if combo_num >= 2:
        # 2. 條件熵 H(Y|X) = Σ p(X = x)*H(Y | X = x)
        conditional_entropy: float = _get_conditional_entropy(
            pUCC_indices=historical_syn_combos_indices,
            syn_data=syn_data_subsets,
            syn_n_rows=syn_n_rows,
            add_column=add_column,
        )

        # 3. 互資訊 I(X; Y)（實際熵 - 條件熵）
        mutual_information: float = column_combo_entropy - conditional_entropy

        # 4. 剪枝：如果互資訊小於等於 0，則繼承自這個欄位組合的欄位不需要繼續執行
        pruning: bool = True if mutual_information <= 0 else False

        results[column_combos].update(
            {
                "conditional_entropy": conditional_entropy,
                "mutual_information": mutual_information,
                "pruning": pruning,
            }
        )

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-end",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # print(column_combos, type(column_combos))
    if set(
        (
            "native-country",
            "fnlwgt",
            "age",
        )
    ) == set(column_combos):
        break

    if add_column == "occupation":
        break

全部進度

In [ ]:
counted_syn_column_combos

set()

In [ ]:
{
    k: {
        key: value
        for key, value in v.items()
        if key
        not in [
            "unique_combos_indices",
            "append_syn_combos_indices",
        ]
    }
    for k, v in results.items()
}

{}

In [ ]:
current_step_combos

[(('capital-loss',), None)]

In [ ]:
config: dict[str, Any] = {}

eval = MPUCCs(config=config)
eval_results = eval.eval(data=eval_dict)
eval_results["details"]

mpUCCs 分析進度:   5%|▌         | 1789/32766 [07:56<2:17:30,  3.75it/s, 大小:5 | 進度:3320/32766 | 剩餘:29446 | 識別:24433/39073(62.5%) | 去重:0]


KeyboardInterrupt: 

In [ ]:
from typing import Optional

eval = MPUCCs(config=config)
syn_column_info = eval._get_column_info(eval_dict["syn"], eval.config["precision"])
sorted_columns: list[str] = [info["colname"] for info in syn_column_info]
sorted_all_combinations: dict[tuple[str], Optional[tuple[str]]] = (
    eval._get_sorted_combinations(
        sorted_columns,
        max_combination_size=eval.config["max_combination_size"],
    )
)

ori_data = eval._preprocess_data(eval_dict["ori"], syn_column_info)
syn_data = eval._preprocess_data(eval_dict["syn"], syn_column_info)

AttributeError: 'MPUCCs' object has no attribute '_get_sorted_combinations'

In [ ]:
from collections import Counter
from typing import Any, Optional

from scipy.stats import entropy
from tqdm import tqdm


def _main_pbar_update(
    main_pbar: tqdm,
    type: str,  # 'loop-start' or 'loop-end'
    main_pbar_info: dict[str, Any],
    combo_num: int = None,
    column_combos: frozenset[str] = None,
    singlingout_syn_indices_num: int = 0,
    syn_n_rows: int = 0,
) -> dict[str, Any]:
    add_column: str = main_pbar_info.get("add_column", None)
    column_names: str = main_pbar_info.get("column_names", None)
    source_combos_num: int = main_pbar_info.get("source_combos_num", None)
    dest_combos_num: int = main_pbar_info.get("dest_combos_num", None)
    all_combos_num: int = main_pbar_info.get("all_combos_num", None)
    if type == "loop-start":
        if combo_num is None or column_combos is None:
            raise ValueError(
                "combo_num/column_combos cannot be None when type is 'loop-start'"
            )

        if combo_num == 1:
            source_combos_num += 1
            dest_combos_num = 0
            all_combos_num = sum(
                [math.comb(source_combos_num - 1, i) for i in range(source_combos_num)]
            )
        dest_combos_num += 1
        main_pbar.set_description_str(
            f"現正處理: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )
    elif type == "loop-end":
        main_pbar.update(1)
        main_pbar.set_description_str(
            f"處理完畢: 第 {str(source_combos_num + 1)} 個欄位 {add_column} 的第 {str(dest_combos_num)} / {str(all_combos_num)} 組合 | "
            f"欄位: {column_names} | "
            f"已識別行數: {singlingout_syn_indices_num}/{syn_n_rows}"
        )

    return {
        "add_column": add_column,
        "source_combos_num": source_combos_num,
        "dest_combos_num": dest_combos_num,
        "all_combos_num": all_combos_num,
    }


def _unique_and_colliding(
    column_combos: tuple[str],
    ori_data_subsets: pd.DataFrame,
    syn_combo_counter: Counter,
    counted_syn_column_combos: set[tuple[str]],
    singlingout_syn_indices: set[int],
    singlingout_syn_indices_num: int,
) -> tuple[
    bool,
    Optional[dict[str, Any]],
    Counter,
    set[tuple[str]],
    set[int],
    int,
]:
    # 只保留出現一次的組合
    all_syn_combos_count: int = len(syn_combo_counter)
    updated_syn_combo_counter: Counter = {
        k: v for k, v in syn_combo_counter.items() if v == 1
    }

    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 在原始資料計算計數，並只保留出現一次的組合
    ori_combo_counter = Counter(map(tuple, ori_data_subsets.values))
    ori_combo_counter = {k: v for k, v in ori_combo_counter.items() if v == 1}

    # 更新合成資料計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in ori_combo_counter
    }
    # 如果計數為空，則跳過本 column_combos
    if not updated_syn_combo_counter:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 此時再來判斷本欄位組合，是否均已被成功碰撞（所有值域組合均唯一、且成功碰撞到唯一原始資料）
    if len(updated_syn_combo_counter) == all_syn_combos_count:
        counted_syn_column_combos.add(column_combos)
        print(f"欄位組合剪枝加入 {column_combos}")

    # 存儲每個組合值對應的行索引
    syn_combo_indices: defaultdict = defaultdict(int)
    for idx, row in syn_data_subsets.iterrows():
        # 同步跳過已經計數過的行
        if idx not in singlingout_syn_indices:
            value_combo = tuple(row)
            # 如果未被跳過、也在本 column_combos 的計數中是唯一
            if value_combo in updated_syn_combo_counter:
                # 納入本 column_combos 的索引
                syn_combo_indices[value_combo] = idx
                # 並添加進已計數
                singlingout_syn_indices.add(idx)
                singlingout_syn_indices_num += 1

    # 如果篩選完沒有合成索引，則跳過本 column_combos
    if not syn_combo_indices:
        return (
            False,
            None,
            None,
            counted_syn_column_combos,
            singlingout_syn_indices,
            singlingout_syn_indices_num,
        )

    # 反向將未被跳過的行索引，同步到合成資料的唯一計數
    updated_syn_combo_counter = {
        k: v for k, v in updated_syn_combo_counter.items() if k in syn_combo_indices
    }
    # 再同步到原始資料的唯一計數
    ori_combo_counter = {
        k: v for k, v in ori_combo_counter.items() if k in syn_combo_indices
    }

    # 最後再紀錄原始資料的行索引
    ori_combo_indices: defaultdict = defaultdict(int)
    for idx, row in ori_data_subsets.iterrows():
        value_combo = tuple(row)
        if value_combo in ori_combo_counter:
            ori_combo_indices[value_combo] = idx

    return (
        True,
        {
            "unique_combos_count": len(syn_combo_indices),
            "unique_combos_indices": {
                id: {
                    "ori": ori_combo_indices.get(id),
                    "syn": syn_combo_indices.get(id),
                }
                for id in set(ori_combo_indices) | set(syn_combo_indices)
            },
        },
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )


def _get_conditional_entropy(
    pUCC_indices: set[int],
    syn_data: pd.DataFrame,
    syn_n_rows: int,
    add_column: str,
) -> float:
    # cnt_pUCC: int = len(pUCC_indices)
    cnt_pUCC: int = len(pUCC_indices)
    cnt_non_pUCC: int = syn_n_rows - cnt_pUCC

    # 2. 計算概率 p(X = mpUCC) 和 p(X ≠ mpUCC)
    p_pUCC: float = len(pUCC_indices) / syn_n_rows
    p_non_pUCC: float = 1 - p_pUCC

    # 3. 計算條件熵 H(Y|X = mpUCC)
    if cnt_pUCC > 0:
        y_given_pUCC = (
            syn_data.loc[list(pUCC_indices), add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_pUCC = entropy(y_given_pUCC)
    else:
        H_Y_given_pUCC = 0

    # 4. 計算條件熵 H(Y|X ≠ mpUCC)
    if cnt_non_pUCC > 0:
        non_pUCC_indices = ~syn_data.index.isin(pUCC_indices)
        y_given_non_pUCC = (
            syn_data.loc[non_pUCC_indices, add_column]
            .value_counts(normalize=True)
            .values
        )
        H_Y_given_non_pUCC = entropy(y_given_non_pUCC)
    else:
        H_Y_given_non_pUCC = 0

    # 5. 計算總條件熵 H(Y|X) = p(X = mpUCC) * H(Y|X = mpUCC) + p(X ≠ mpUCC) * H(Y|X ≠ mpUCC)
    H_Y_given_X = p_pUCC * H_Y_given_pUCC + p_non_pUCC * H_Y_given_non_pUCC
    return H_Y_given_X


# 計算筆數
syn_n_rows: int = syn_data.shape[0]
# 計算總組合數
total_combinations_num: int = len(sorted_all_combinations)
# 設置主進度條
main_pbar = tqdm(
    total=total_combinations_num,
    desc="全部進度",
    bar_format="{desc}",
    position=0,
    leave=True,
)


main_pbar_info: dict[str, Any] = {
    "add_column": None,
    "column_names": None,
    "source_combos_num": 0,
    "dest_combos_num": 0,
    "all_combos_num": 0,
}
singlingout_syn_indices_num = 0
counted_syn_column_combos: set[tuple[str]] = set()
singlingout_syn_indices: set[int] = set()
singlingout_syn_indices_num: int = 0

results: dict[str, dict[str, Any]] = {}
for column_combos, source_column_combos in sorted_all_combinations.items():
    combo_num: int = len(column_combos)
    column_names: str = ", ".join(str(col) for col in column_combos)
    main_pbar_info["column_names"] = column_names
    results[column_combos] = {}

    # 捕捉本批新增的欄位
    if combo_num == 1:
        add_column: str = next(iter(column_combos))
        main_pbar_info["add_column"] = add_column

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-start",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # 各種早停規則
    skip_tag: bool = False
    break_tag: bool = False
    # stop1. 跳過已被
    if results.get(source_column_combos, {}).get("pruning", False):
        print(f"互資訊無增長剪枝觸發，跳過 {column_names}")
        results[column_combos]["pruning"] = True  # 持續剪枝
        skip_tag = True
    # stop2. 終止於所有行都已被唯一計數
    if singlingout_syn_indices_num == syn_n_rows:
        print(
            f"所有行已被指認剪枝觸發，終止於 {', '.join(str(col) for col in source_column_combos)}"
        )
        break_tag = True
    # stop3. 跳過已經唯一計數子集的超集
    if any(
        set(counted_combo).issubset(column_combos)
        for counted_combo in counted_syn_column_combos
    ):
        print(f"欄位組合剪枝觸發，跳過 {column_names}")
        skip_tag = True

    if break_tag:
        break
    if skip_tag:
        main_pbar.update(1)
        continue

    # 整理資料
    ori_data_subsets: pd.DataFrame = ori_data.loc[:, column_combos]
    syn_data_subsets: pd.DataFrame = syn_data.loc[:, column_combos]

    # 1. 計算欄位組合的熵 H(XY)
    syn_combo_counter: Counter = Counter(map(tuple, syn_data_subsets.values))
    column_combo_entropy: float = entropy(
        [(count / syn_n_rows) for count in syn_combo_counter.values()]
    )
    results[column_combos]["column_combo_entropy"] = column_combo_entropy

    # > 唯一且碰撞
    (
        unique_colliding_sts,
        unique_syn_combos_info,
        updated_syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    ) = _unique_and_colliding(
        column_combos,
        ori_data_subsets,
        syn_combo_counter,
        counted_syn_column_combos,
        singlingout_syn_indices,
        singlingout_syn_indices_num,
    )
    # 如果沒有唯一且碰撞的組合，則跳過
    if not unique_colliding_sts:
        main_pbar.update(1)
        continue
    results[column_combos].update(unique_syn_combos_info)

    # 捕捉本次合成資料的行索引
    append_syn_combos_indices: set[int] = {
        idx
        for indices_pair in results[column_combos]["unique_combos_indices"].values()
        for data_type, idx in indices_pair.items()
        if data_type == "ori"
    }
    # 並添加上累積的行索引
    historical_syn_combos_indices: set[int] = results.get(source_column_combos, {}).get(
        "append_syn_combos_indices", set()
    )
    append_syn_combos_indices = (
        append_syn_combos_indices | historical_syn_combos_indices
    )
    results[column_combos]["append_syn_combos_indices"] = append_syn_combos_indices

    if combo_num >= 2:
        # 2. 條件熵 H(Y|X) = Σ p(X = x)*H(Y | X = x)
        conditional_entropy: float = _get_conditional_entropy(
            pUCC_indices=historical_syn_combos_indices,
            syn_data=syn_data_subsets,
            syn_n_rows=syn_n_rows,
            add_column=add_column,
        )

        # 3. 互資訊 I(X; Y)（實際熵 - 條件熵）
        mutual_information: float = column_combo_entropy - conditional_entropy

        # 4. 剪枝：如果互資訊小於等於 0，則繼承自這個欄位組合的欄位不需要繼續執行
        pruning: bool = True if mutual_information <= 0 else False

        results[column_combos].update(
            {
                "conditional_entropy": conditional_entropy,
                "mutual_information": mutual_information,
                "pruning": pruning,
            }
        )

    # 更新資訊顯示
    main_pbar_info = _main_pbar_update(
        main_pbar,
        type="loop-end",
        main_pbar_info=main_pbar_info,
        combo_num=combo_num,
        column_combos=column_combos,
        singlingout_syn_indices_num=singlingout_syn_indices_num,
        syn_n_rows=syn_n_rows,
    )

    # print(column_combos, type(column_combos))
    if set(
        (
            "native-country",
            "fnlwgt",
            "age",
        )
    ) == set(column_combos):
        break

    if add_column == "occupation":
        break

全部進度

In [ ]:
counted_syn_column_combos

set()

In [ ]:
{
    k: {
        key: value
        for key, value in v.items()
        if key
        not in [
            "unique_combos_indices",
            "append_syn_combos_indices",
        ]
    }
    for k, v in results.items()
}

{}